In [2]:
import numpy as np
from math import pi, sqrt
from scipy import linalg as LA
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib import ticker, cm
import cmath


##########################################################################################
# Chebyshev Interpolation routines
##########################################################################################


def Chebyshev(m,x) :
    
    """
    Returns the value of the T_m(x)
    """
    return cos(m*acos(x))


def Cheb_spectral_interpol(coeffs,x) :

    """
    Returns the Chebyshev interpolation at x\in[1,1], given the
    Chebishev coefficients coeffs
    """
    n = len(coeffs)
    N = n-1
    n_grid = np.arange(0,n)
    
    temp=0
    for k in np.arange(1,n):
        temp += coeffs[k]*Chebyshev(k,x)
    
    interpol = 1./2.*coeffs[0] + temp
    
    return interpol

##########################################################################################
# Chebyshev basic spectral elements
##########################################################################################

##########################################################################################
#Chebyshev-Lobatto
##########################################################################################

def CL_grid(N) : 
    
    """
    Implement collocation points of the Chebyshev-Lobatto grid
    x_j with j=0,1,...,N
    (following point 4.6.4. in Marcus' notes)
    
    It returns a tuple with the grid points in the
    first entry and the Differentation Matrix in the second entry
    """
    n = N+1
    n_grid = np.arange(0, n)
    x_CL = np.cos(pi*n_grid/N)
    
    return x_CL

def coeffs_CL(f) :
    
    """ 
    Reads an ndarray with the values of the function at the Chebyshev-Lobatto
    collocation points and return the spectral coefficients
    (following point 4.3.4. in Marcus' notes)
    
    Note: f[N] corresponds to f(-1) and f[0]=1
    """
    
    n = len(f)
    N = n-1
    n_grid = np.arange(0,n)
    
    coeffs = np.zeros(n)
    print(coeffs)
    

    for i in n_grid :
        if i == N :
            temp = 0.
            for k in np.arange(1,N) :
                temp += f[k]*Chebyshev(i,CL_grid(N)[k])
            coeffs[i] = 1./(2.*N)*(f[0] + np.float_power((-1),i)*f[N] + 2.*temp)
            
        else :
            temp = 0
            for k in np.arange(1,N) :
                temp +=  f[k]*Chebyshev(i,CL_grid(N)[k])
            coeffs[i] = 1./N*(f[0] + np.float_power((-1),i)*f[N] + 2.*temp)
            
    return coeffs
    


##########################################################################################
# Spectral-Derivation Matrices
##########################################################################################

##########################################################################################
#Chebyshev-Lobatto
##########################################################################################


def D1_CL(N) :

    """
    Implement the matrix of the 1st Derivative
    associated with the Chebyshev-Lobatto grid,
    with grid points: x_j with j=0,1,...,N
    (following point 4.6.4. in Marcus' notes)
    
    It returns a tuple with the grid points in the
    first entry and the Differentation Matrix in the second entry
    """
    
    n = N+1
    n_grid = np.arange(0, n)
    x_CL = np.cos(pi*n_grid/N)
    
    kappa = np.ones(n); kappa[0]=2.; kappa[N]=2.
    D1_CL = np.zeros((n,n))

    for i in n_grid :
        #Diagonal values
        D1_CL[0,0] = (2.*N**2+1.)/6.
        D1_CL[N,N] = -(2.*N**2+1.)/6.
        if ((i!=0) & (i!=N)) : D1_CL[i,i] = -x_CL[i]/(2*(1-x_CL[i]**2))
        #Out-of-diagonal values
        for j in n_grid :
            if (j!=i) : D1_CL[i,j] = kappa[i]/kappa[j]* (-1.)**(i-j)/(x_CL[i]-x_CL[j])
            #if (j!=i) : D1_CL[i,j] = (-1)**(i-j)    
    
    return x_CL, D1_CL


    
def D2_CL(N) :

    """
    Implement the matrix of the 2nd Derivative
    associated with the Chebyshev-Lobatto grid,
    with grid points: x_j with j=0,1,...,N
    (following point 4.6.4. in Marcus' notes)
    
    It returns a tuple with the grid points in the
    first entry and the Differentation Matrix in the second entry
    """
    
    n = N+1
    n_grid = np.arange(0, n)
    x_CL = np.cos(pi*n_grid/N)
    
    kappa = np.ones(n); kappa[0]=2.; kappa[N]=2. 
    D2_CL = np.zeros((n,n))

    for i in n_grid :
        #Diagonal values
        D2_CL[0,0] = (N**4-1.)/15.
        D2_CL[N,N] = D2_CL[0,0]
        if ((i!=0) & (i!=N)) : 
            D2_CL[i,i] = -1./(1-x_CL[i]**2)**2 - (N**2-1.)/(3.*(1.-x_CL[i]**2))
        #Out-of-diagonal values
        for j in n_grid :
            if (j!=0) : 
                D2_CL[0,j] = 2./3. * (-1.)**j/kappa[j] * ((2.*N**2+1.)*(1-x_CL[j])-6.)/(1.-x_CL[j])**2
            if (j!=N) : 
                D2_CL[N,j] = 2./3. * (-1.)**(N+j)/kappa[j] * ((2.*N**2+1.)*(1+x_CL[j])-6.)/(1.+x_CL[j])**2
            if ((j!=i) & (i!=0) & (i!=N)) : 
                D2_CL[i,j] = (-1.)**(i-j)/kappa[j] * (x_CL[i]**2+x_CL[i]*x_CL[j]-2.)/((1-x_CL[i]**2)*(x_CL[j]-x_CL[i])**2)  

    return x_CL, D2_CL

##########################################################################################
#Chebyshev-Gauss
##########################################################################################


def D1_CG(N) :

    """
    Implement the matrix of the 1st Derivative
    associated with the Chebyshev-Gauss grid,
    with grid points: x_j with j=0,1,...,N
    (following point 4.6.3. in Marcus' notes)
    
    It returns a tuple with the grid points in the
    first entry and the Differentation Matrix in the second entry
    """
    
    n = N+1
    n_grid = np.arange(0, n)
    x_CG = np.cos((pi*n_grid+0.5*pi)/(N+1))
    D1_CG = np.zeros((n,n))

    for i in n_grid :
        for j in n_grid :
            if (i==j) :   D1_CG[i,j] = (x_CG[i])/(2*(1-(x_CG[i])**2))
            if (i!=j) :   D1_CG[i,j]=(np.float_power((-1),(i-j)))/(x_CG[i]-x_CG[j])*sqrt((1-(x_CG[j])**2)/(1-x_CG[i]**2))
    
    return x_CG, D1_CG 


def D2_CG(N) :
    
    """
    Implement the matrix of the 2nd Derivative
    associated with the Chebyshev-Gauss grid,
    with grid points: x_j with j=0,1,...,N
    (following point 4.6.3. in Marcus' notes)
    
    
    It returns a tuple with the grid points in the
    first entry and the Differentation Matrix in the second entry
    """
    
    n = N+1
    n_grid = np.arange(0, n)
    x_CG = np.cos((pi*n_grid+pi*0.5)/(N+1))
    D2_CG = np.zeros((n,n))

    for i in n_grid :
        for j in n_grid :
            if (i==j) : D2_CG[i,j] = (x_CG[i])**2/(1-(x_CG[i])**2)**2-N*(N+2)/(3*(1-(x_CG[i])**2))
            else : D2_CG[i,j] = (np.float_power((-1),(i-j)))/(x_CG[i]-x_CG[j])*sqrt((1-x_CG[j]**2)/(1-x_CG[i]**2))*(((x_CG[i])/(1-x_CG[i]**2))-(2/(x_CG[i]-x_CG[j])))
    return x_CG, D2_CG   


##########################################################################################
# Spectral Analysis Tools
##########################################################################################

def Pseudospectrum(L,B,xmin,xmax,ymin,ymax,Nxgrid,Nygrid,heights,fl,log_norm_Random) :
    """
    Implement Pseudo-Spectrum of a matrix L
    """
    #import numpy as np
    #from math import pi
    from scipy import linalg as LA
    
    ### 1. Preparation of tools 
    sizeL = np.shape(L)
    if  np.shape(L)[0]!= np.shape(L)[1]:
        print("Non-square Matrix!!! ")
    else : 
        n = np.shape(L)[0]
    N = n/2 - 1
   
    ### 2. Calculation of the Spectrum (eigenvalues)
    #eigenvalues_L, eigenvectors_L = LA.eig(L,B)
    eigenvalues_L = LA.eigvals(L,B)


    eigenvalues_L_Re = eigenvalues_L.real
    eigenvalues_L_Im = eigenvalues_L.imag
    
#    for i in range(N+1) :
#        print("lambda = ",eigenvalues_L_Re[i],"+i",eigenvalues_L_Im[i])

    
    ### 3. Evaluation of the Pseudospectrum
    
    ### 3.1 Grid for Pseudospectrum calculation
    [X,Y] = np.mgrid[xmin:xmax:Nxgrid*1j,ymin:ymax:Nygrid*1j]

    Z = X + 1j*Y
    #print("X = ",X, "\n")
    #print("Y = ",Y, "\n")
    #print(Y)
    #print("Z =", Z)
    #print(Z[0,1])
    #print("\n")

    ### 3.2 Construction of the "height function" given by the min of the SVP
    Id =  np.eye(n)
    Sigma_min = np.zeros((Nxgrid,Nygrid))
    

    #print(L)
    #print(Sigma_min)
    #print(Id)

    for i in np.arange(0, Nxgrid):
        for j in np.arange(0, Nygrid):
            L_shift = L - Z[i,j]*B*Id            
            Sigma_min[i,j] = min(np.linalg.svd(L_shift, full_matrices=True)[1]) 

    #lambda_test = 1 + 1j
    #print("lambda_test = ", lambda_test)
    #Test_svd = L - lambda_test*B*Id
    #print("test_min_sv = ", min(np.linalg.svd(Test_svd, full_matrices=True)[1]) )
    
            
    #print("Sigma_min = ", Sigma_min)

    #plt.ion()
    #curves=plt.contour(X,Y,Sigma_min,12)
    #plt.CLabel(curves)
    
    ### 3.3 Graphical output
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(eigenvalues_L_Re, eigenvalues_L_Im, '+', markersize=1)
    #curves=plt.contour(X,Y,Sigma_min,12)
    #ax.CLabel(curves)    
    if fl == "f" :
        CS = ax.contourf(X,Y,Sigma_min,heights,locator=ticker.LogLocator(),linewidth=0.5)
    elif fl == "c" : 
        CS = ax.contour(X,Y,Sigma_min,heights,locator=ticker.LogLocator(),linewidth=0.5)
    else :
        print("\n Pseudospectrum output: \n No 'contour/filled' version could be identified.\n Filled version is assumed.\n")
        CS = ax.contourf(X,Y,Sigma_min,heights,locator=ticker.LogLocator(),linewidth=0.5)
    #CS = ax.contour(X,Y,Sigma_min,12)
    #ax.CLabel(CS)
    #ax.CLabel(plt.contour(X,Y,Sigma_min,heights))
    #ax.CLabel(plt.contour(X,Y,Sigma_min,12))
    CB = fig.colorbar(CS)
    ax.set_xlabel(r'$\mathrm{Re}(\omega_n)$')
    ax.set_ylabel(r'$\mathrm{Im}(\omega_n)$')    
    f = mticker.ScalarFormatter(useOffset=False, useMathText=True)
    g = lambda x,pos : "${}$".format(f._formatSciNotation('%10e' % x))
    ax.xaxis.set_major_formatter(mticker.FuncFormatter(g))
    ax.yaxis.set_major_formatter(mticker.FuncFormatter(g))
    
    fig.suptitle(r'Spectrum and Pseudospectrum of $L$ with $log||\mathrm{Random}||_2=%3g$' % log_norm_Random)
    ax.axis('scaled')
    #ax.axis('equal','datalim')
    ax.axis([xmin,xmax,ymin,ymax])
    ax.grid()   
    #ax.set_xlim(xmin,xmax)
    #ax.set_ylim(ymin,ymax)

    fig.show()
    #fig.savefig("/home/jaramillo/Dropbox/Trabajo/Programacion/Python/Diagonalization/Eigenvalues_L.pdf")
    print("\n N =\n",N)
    #fig.savefig("Figures/Pseudospectrum_Rn"+str(log_norm_Random)+"_"+str(N+1)+".pdf")
    fig.savefig("Figures/Pseudospectrum_Rn"+str(log_norm_Random)+"_N"+str(N)+"_Nx"+str(Nxgrid)+"_Ny"+str(Nygrid)+".pdf")
    



    

##########################################################################################
# Miscellanea Analysis Tools
##########################################################################################

    
def matrice_random(n) :
    R = np.random.rand(n,n)
    return R

def matrice_random_complex(n) :
    R1 = np.random.rand(n,n)
    R2 = np.random.rand(n,n)
    R = R1 + R2*1j
    return R

def Diagonal_Random_Matrix(n) :
    D = np.random.rand(n,1)
    Id = np.eye(n)
    DR = Id * D 
    #visualisation_matrix(DR)
    #DR = np.zeros(n,n)    
    #for i in range(n):
    #    DR[i,i] = D[i,i] 
    return DR

def TriDiagonal_Random_Matrix(n) :
    Dabove = np.random.rand(n-1,1)
    Dprincipal = np.random.rand(n,1)
    Dbelow = np.random.rand(n-1,1)
    DR = np.zeros((n,n))    
    for i in range(n):
        if i!=n-1: DR[i,i+1] = Dabove[i,0]
        DR[i,i] = Dprincipal[i,0]
        if i!=n-1: DR[i+1,i] = Dbelow[i,0]
    return DR

def visualisation_matrix(A):
    "Qualitative visualization of the matrix"
    
    from matplotlib import pyplot as plt
    plt.close()
    plt.imshow(A, interpolation='nearest', cmap=plt.cm.gray_r)
    plt.colorbar()
    plt.show()


def norm_2(A) :
    sigma_max = max(np.linalg.svd(A, full_matrices=True)[1])
    return sigma_max

#Diagonal_Random_Matrix(5)
